In [21]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk

# Load captions
captions_file = "C:/Users/ewang/OneDrive/Desktop/Spring_2024/COS_429/Final/captions_GIT_20_percent.csv"
captions = pd.read_csv(captions_file)
light_captions = captions["light_caption"]
dark_captions = captions["dark_caption"]

# Load top 100 words
top100_file = "C:/Users/ewang/OneDrive/Desktop/Spring_2024/COS_429/Final/imagecaptioning-bias/annotations/top100_overall.txt"
with open(top100_file, 'r') as file:
    top_100_words = file.read().splitlines()

nltk.download('stopwords')
stop_words = list(set(stopwords.words('english')))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ewang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:

# Create feature vectors for light and dark captions
# vectorizer = CountVectorizer(vocabulary=top_100_words, binary=True)
vectorizer = CountVectorizer(max_features=100, stop_words=stop_words)
all_captions = pd.concat([light_captions, dark_captions])
vectorizer.fit(all_captions)
X_light = vectorizer.transform(light_captions).toarray()
X_dark = vectorizer.transform(dark_captions).toarray()

# Combine features and create labels
X = pd.concat([pd.DataFrame(X_light), pd.DataFrame(X_dark)], ignore_index=True)
y = [0] * len(light_captions) + [1] * len(dark_captions)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [23]:
# Predict and evaluate on the test set
y_pred = model.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred)
print(f"AUC Score: {auc_score}")

# Show the most important words based on the trained model
feature_names = vectorizer.get_feature_names_out()
feature_importance = model.coef_[0]
important_words = sorted(zip(feature_importance, feature_names), key=lambda x: x[0], reverse=True)
print("Top 10 words for Dark skin tone captions:")
for coef, word in important_words[:10]:
    print(f"{word}: {coef}")
print("Top 10 words for Light skin tone captions:")
for coef, word in important_words[-10:]:
    print(f"{word}: {coef}")

AUC Score: 0.5475806451612903
Top 10 words for Dark skin tone captions:
men: 1.0855092196945184
wearing: 1.0058520112438547
playing: 0.8645119548770718
green: 0.7890974164791461
ocean: 0.7353292775544883
sitting: 0.694284983172524
racket: 0.6906512378398689
grass: 0.6730497932637167
two: 0.6389334864216434
bus: 0.6345302168633518
Top 10 words for Light skin tone captions:
tie: -0.5010401729960112
horse: -0.5240866703178665
laying: -0.5357575284526603
large: -0.5571842123899241
plate: -0.5646614221202444
beard: -0.5667925942044456
black: -0.5809127624572427
swinging: -0.7884249698978177
group: -0.8783693367440539
little: -1.3067677164189764
